# Data Collection

In [7]:
# Get general dependencies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Import college football api
!pip install --q cfbd python-dotenv

In [3]:
# Install API from college football data
import cfbd
from dotenv import dotenv_values

# Load stuff from .env file
env_vars = dotenv_values('.env')

configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = env_vars.get('CFBD_API_KEY')
configuration.api_key_prefix['Authorization'] = env_vars.get('CFBD_API_KEY_PREFIX')

api_config = cfbd.ApiClient(configuration)

In [16]:
# Create a teams api instance and a games api instance and explore
#teams_api = cfbd.TeamsApi(api_config)
games_api = cfbd.GamesApi(api_config)
#players_api = cfbd.PlayersApi(api_config)

In [28]:
# Get team records
def get_records(**kwargs):
  records = []
  for year in kwargs['years']:
    for team in kwargs['teams']:    
      records += kwargs['api'].get_team_records(year=year, team=team)
  return records

# Parse the plays into a dataframe
def records_to_df(records): 
  records_dict = [dict(
    team = r.team,
    team_id = r.team_id,
    conference = r.conference,
    games_played = r.total.games,
    wins = r.total.wins,
    losses = r.total.losses,
  ) for r in records if r.team is not None]
  records_df = pd.DataFrame(records_dict)
  return records_df 

In [29]:
#games_api.get_team_records(year=2023, team='South Carolina')
records_df = records_to_df(
    get_records(api=games_api, years=[2023], teams=['South Carolina'])
)
records_df.head()

,team,team_id,conference,games_played,wins,losses
0,South Carolina,2579,SEC,12,5,7


In [27]:
games_api.get_team_records(year=2023, team='South Carolina')

[{'away_games': {'games': 4, 'losses': 4, 'ties': 0, 'wins': 0},
  'conference': 'SEC',
  'conference_games': {'games': 8, 'losses': 5, 'ties': 0, 'wins': 3},
  'division': 'East',
  'expected_wins': 5.6,
  'home_games': {'games': 7, 'losses': 2, 'ties': 0, 'wins': 5},
  'team': 'South Carolina',
  'team_id': 2579,
  'total': {'games': 12, 'losses': 7, 'ties': 0, 'wins': 5},
  'year': 2023}]